In [1]:
! pip install stop_words

You are using pip version 8.0.3, however version 8.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = {'s1': s1, 's2': s2, 's3': s3, 's4': s4}

In [3]:
len(docs)

4

**Function to make Corpus Matrix**

Function returns a list, 0th element is a 3 dimensional array, each element corresponds to a document in the corpus where the columns are the unique words in that document and the rows are the words in that document (stop words removed).  Second element in list is the column names for each document.

In [4]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words

stopWords = get_stop_words('english')

#Function to make document, word matricies for LDA#
def make_word_matrix(corpus):
    D = len(corpus)
    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))
   
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        #Remove punctuation 
        text = docs[i].translate(table)
        #Splits string by blankspace and goes to lower case#
        words = text.lower().split()
    
        #Remove stop words#
        text = [word for word in words if word not in stopWords]
        
        #Find total number of words in each document#
        N = len(text)
        
        #Find number of unique words in each document#
        V = len(set(text))
        Vwords = list(set(text))
        wordOrder.append(Vwords)
        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V))
        count = 0
        for word in text:
            v = Vwords.index(word)
            wordsMat[count, v] = 1
            count = count + 1
        c.append(wordsMat)

    return [c, wordOrder]



In [5]:
corpusMatrix = make_word_matrix(docs)
corpusMatrix[0][0].shape

(5, 5)

**Variational Inference**

In [ ]:
import numpy as np
import scipy
from scipy import special

In [ ]:
#lengths and number of topics will go here

k = 5
D = 4
tol = 1

#need to define number of documents and number of topics here, 

#looping through the number of documents
for d in range(0,D): #D is the number of documents
    
    #storing the total number of words and the number of unique words
    N = corpusMatrix[0][d].shape[0]
    V = corpusMatrix[0][d].shape[1]
    
    #initialize phi and gamma
    alpha = np.full(shape = k, fill_value = 1)
    phi = np.full(shape = (N,k), fill_value = 1/k)
    beta = np.full(shape = (k, V), fill_value = 1/k)
    gamma = alpha + N/k
    converge = 0 
    
    
    
    while converge == 0:
    
        #creating a place to store the updated phi
        newPhi  = np.full(shape = (N,k), fill_value = 1/k)
        for n in range(0, N):
            for i in range(0,k):
                newPhi[n,i] = (beta[i, list(corpusMatrix[0][d][n,:]).index(1)])*scipy.special.psi(gamma[i])
        newPhi = newPhi/np.sum(newPhi, axis = 1) #normalizing the rows of new phi
        newGamma = alpha + np.sum(newPhi) #updating gamma
        
        #checking for convergence
        if np.linalg.norm(newGamma - gamma) + np.linalg.norm(newPhi - phi) < tol:
            converge = 1
        else:
            converge = 0
    